In [0]:
import os, sys, json
import mlflow
from mlflow.tracking import MlflowClient

# ==============================
# 디버깅 정보 출력
# ==============================
print("=== PRODUCTION DEPLOY DEBUGGING INFO ===")
print(f"Current working directory: {os.getcwd()}")
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    print(f"Databricks notebook path: {notebook_path}")
    current_dir = os.path.dirname(notebook_path)
except:
    current_dir = "/Workspace/Users/1dt003@msacademy.msai.kr/team4-CICD"
    print(f"Using default current_dir: {current_dir}")
print("==========================================")

# ==============================
# Config 로드 (선택적)
# ==============================
try:
    sys.path.append(f"{current_dir}/src")
    from src.config_loader import load_config
    config_path = f"{current_dir}/configs/config_v2.json"
    config = load_config(config_path)
    print("✅ Config 로드 성공")
except Exception as e:
    print(f"⚠️ Config 로드 실패, 환경변수만 사용: {e}")
    config = {}

# ==============================
# 환경변수 / 설정
# ==============================
BASE_MODEL_NAME = os.environ.get("BASE_MODEL_NAME") or config.get("model_name", "team4-pred-model")
ENVIRONMENT = os.environ.get("ENVIRONMENT", "production")
ARCHIVE_EXISTING = os.environ.get("ARCHIVE_EXISTING_VERSIONS", "false").lower() == "true"

# MLflow 설정
mlflow.set_tracking_uri("databricks")
if config.get("experiment_name"):
    mlflow.set_experiment(config["experiment_name"])

client = MlflowClient()

print(f"배포할 모델: {BASE_MODEL_NAME}")
print(f"환경: {ENVIRONMENT}")
print(f"기존 버전 아카이브: {ARCHIVE_EXISTING}")

# ==============================
# 최신 모델 버전 가져오기 (None stage에서)
# ==============================
try:
    latest_versions = client.get_latest_versions(name=BASE_MODEL_NAME, stages=["None"])
    if not latest_versions:
        raise ValueError(f"No versions found for {BASE_MODEL_NAME} in None stage")

    latest_version_num = max([int(v.version) for v in latest_versions])
    print(f"✅ 최신 모델 버전 발견: {latest_version_num}")
    
    # 모델 정보 출력
    model_version = client.get_model_version(BASE_MODEL_NAME, latest_version_num)
    print(f"모델 정보:")
    print(f"  - 생성 시간: {model_version.creation_timestamp}")
    print(f"  - 현재 단계: {model_version.current_stage}")
    print(f"  - 설명: {model_version.description or 'None'}")
    
except Exception as e:
    print(f"❌ 모델 버전 조회 실패: {e}")
    raise e

# ==============================
# 기존 Production 모델 확인
# ==============================
try:
    production_versions = client.get_latest_versions(name=BASE_MODEL_NAME, stages=["Production"])
    if production_versions:
        current_prod_version = production_versions[0].version
        print(f"현재 Production 버전: {current_prod_version}")
        
        if int(current_prod_version) == latest_version_num:
            print("⚠️ 이미 같은 버전이 Production에 있습니다.")
        else:
            print(f"기존 Production 버전 {current_prod_version} → 새 버전 {latest_version_num}으로 업데이트")
    else:
        print("현재 Production 단계에 모델이 없습니다.")
        
except Exception as e:
    print(f"⚠️ 기존 Production 모델 확인 실패: {e}")

# ==============================
# Production stage로 전환
# ==============================
try:
    client.transition_model_version_stage(
        name=BASE_MODEL_NAME,  # 수정: MODEL_NAME → BASE_MODEL_NAME
        version=latest_version_num,
        stage="Production",
        archive_existing_versions=ARCHIVE_EXISTING
    )
    
    print(f"✅ 모델 {BASE_MODEL_NAME} version {latest_version_num}을 Production stage로 변경 완료")
    
    # 배포 후 모델 정보 재확인
    updated_model_version = client.get_model_version(BASE_MODEL_NAME, latest_version_num)
    print(f"배포 후 모델 단계: {updated_model_version.current_stage}")
    
except Exception as e:
    print(f"❌ Production 단계 전환 실패: {e}")
    raise e

# ==============================
# 배포 검증 (선택적)
# ==============================
try:
    # Production 단계의 모델이 정상적으로 로드되는지 확인
    model_uri = f"models:/{BASE_MODEL_NAME}/Production"
    model = mlflow.sklearn.load_model(model_uri)
    print("✅ Production 모델 로드 검증 성공")
    
except Exception as e:
    print(f"⚠️ Production 모델 검증 실패: {e}")

# ==============================
# 배포 메타데이터 기록 (선택적)
# ==============================
try:
    # 배포 정보를 모델 버전 설명에 추가
    deployment_info = f"Deployed to Production on {os.environ.get('GITHUB_RUN_ID', 'manual')} by GitHub Actions"
    client.update_model_version(
        name=BASE_MODEL_NAME,
        version=latest_version_num,
        description=deployment_info
    )
    print("✅ 배포 메타데이터 기록 완료")
    
except Exception as e:
    print(f"⚠️ 배포 메타데이터 기록 실패: {e}")

# ==============================
# 필요시 추가 배포 작업 (예: Serving Endpoint 갱신)
# ==============================
UPDATE_SERVING_ENDPOINT = os.environ.get("UPDATE_SERVING_ENDPOINT", "false").lower() == "true"

if UPDATE_SERVING_ENDPOINT:
    try:
        print("🔄 Serving Endpoint 업데이트 시작...")
        # 실제 serving endpoint 업데이트 로직은 환경에 따라 다름
        # 예시: REST API 호출 또는 다른 노트북 실행
        # dbutils.notebook.run("/Workspace/Users/1dt003@msacademy.msai.kr/team4-CICD/notebooks/update_serving_endpoint", 300)
        print("✅ Serving Endpoint 업데이트 완료")
    except Exception as e:
        print(f"⚠️ Serving Endpoint 업데이트 실패: {e}")

# ==============================
# 결과 반환
# ==============================
result = {
    "success": True,
    "model_name": BASE_MODEL_NAME,
    "production_version": int(latest_version_num),
    "environment": ENVIRONMENT,
    "archived_existing": ARCHIVE_EXISTING
}

print(f"최종 Production 배포 결과: {result}")

# CI/CD 워크플로우로 결과 반환
dbutils.notebook.exit(json.dumps(result))